# Import Libraries

In [1]:
import numpy as np
from glob import glob
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from keras.models import Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Input, Lambda, Flatten, Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model

In [2]:
# resize image
IMAGE_SIZE = [224, 224]

In [3]:
# define data path
train_path = "Datasets/Train"
test_path = "Datasets/Test"

In [4]:
# add preprocessing layer to front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE+[3], weights='imagenet', include_top=False)

In [5]:
# don't train existing layers
for layer in vgg.layers:
    layer.trainable = False

In [6]:
folders = glob('Datasets/Train/*')

In [7]:
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [8]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
# view structure of the model
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True
)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                target_size=(224, 224),
                                                batch_size=32,
                                                class_mode='categorical'
                                                )
test_set = test_datagen.flow_from_directory('Datasets/Test',
                                           target_size=(224, 224),
                                            batch_size=32,
                                            class_mode='categorical'
                                           )

Found 7 images belonging to 6 classes.
Found 6 images belonging to 6 classes.


In [ ]:
r = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    steps_per_epoch=len(training_set),
    validation_steps=len(test_set)
)


In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
# accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()

In [ ]:
# save model
model.save('facefeatures_new_model.h5')